In [ ]:
from ddm.data_handling import read_file
from ddm.processing import ddm_numpy #need to make sure we use the proper ddm function with dask functionality
from ddm.fitting import findMeanSqFFT_np, computeAB
from ddm.plotting import plotSingleExpFit, plotISF
import numpy as np
import matplotlib.pyplot as pl
from tqdm.notebook import tqdm

Begin by loading in data<br/>
Run the ddm function on the loaded dataset<br/>
From the resultant g(q, tau), determine A and B using findMeanSqFFT and computeAB - at this point move away from dask arrays.<br/>
Determine the ISF for various q values<br/>
Run fitting procedures on these with a few models and show the output<br/>

Compare to traditional particle tracking algorithms

**Loading in Data**<br/>
At this point a user input may be required if the programme detects a multi-experiment file

In [ ]:
filename = '../data/test_dataset.tif'

ddmData = read_file(filename)

In [ ]:
ddmData.data

**Running the DDM Function**<br/>
This part will determine the full DDM matrix as the square of the difference between Fourier transformed frames

In [ ]:
ddmData = ddmData.data[0:500,:,:]
ddmData = np.asarray(ddmData)


In [ ]:
ddmData.shape

In [ ]:
ddmMatrix = ddm_numpy(ddmData, 1)

In [ ]:
ddmMatrix.shape

In [ ]:
intTaus = np.arange(len(ddmData)//2)
# ddmMatrix = np.array([None for j in range(len(intTaus))])
ddmMatrix = []
for tau in tqdm(intTaus):
    # ddmMatrix[tau] = ddm_numpy(ddmData, tau) #at this point we have the 0th axis as taus and the 1st axis as qs
    ddmMatrix.append(ddm_numpy(ddmData, tau))
ddmMatrix = np.asarray(ddmMatrix)

In [ ]:
pl.plot(ddmMatrix[10,:])
pl.plot(ddmMatrix[25,:])
pl.plot(ddmMatrix[49,:])
pl.xlim(0, 80)

**Determining A and B**

In [ ]:
meanSqFFT = findMeanSqFFT_np(ddmData)
A, B = computeAB(meanSqFFT)

In [ ]:
pl.plot(A[10:])
pl.axhline(B)

**Calculating the ISF**

In [ ]:
isf = 1. - (ddmMatrix - B)/A

isf = np.transpose(isf)

To access a specific q value (in units of 1/pixels) one can use regular indexing of the 0th axis of the ISF

In [ ]:
#zero q
isf[0]

#medium q
print(isf[len(isf)//2])

#last q
isf[len(isf)-1]

**Fitting/Plotting**<br/>
An example to fit and plot to a single exponential. Supported fitting functions are double exponential and the Schultz model

In [ ]:
# taus = intTaus*ddmData.tScale #taus in milliseconds
taus = intTaus
plotISF(isf[:,1:], taus[1:], q = len(isf)//2)
pl.show()

In [ ]:
isf.shape